In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from sklearn.model_selection import train_test_split
device = 'cuda'
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
import time

2023-08-11 13:39:13.442596: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 13:39:13.513390: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-11 13:39:13.529748: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-11 13:39:13.791132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
tokenizer = BertTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru", do_lower_case=True)

In [3]:
model = BertForSequenceClassification.from_pretrained('ai-forever/sbert_large_nlu_ru')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=

In [5]:
for param in model.parameters():
    param.requires_grad = False



In [6]:
model.classifier = nn.Linear(1024, 18)
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.weight
	 classifier.bias


In [7]:
optimizer = AdamW(params_to_update, lr = 2e-4, eps = 1e-8)

/home/seb/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for batch in data_iter:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                  'token_type_ids':batch[2],

                     }
        outputs =net(**inputs)
        acc_sum += (outputs[0].argmax(axis=1) == batch[3]).sum()
        n += batch[3].shape[0]
    return acc_sum.item() / n

In [9]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, batch in enumerate(train_iter):
            train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                      'token_type_ids':batch[2],

                         }
            trainer.zero_grad()
            outputs =net(**inputs)
            l = loss(outputs[0], batch[3])
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (outputs[0].argmax(axis=1) == batch[3]).sum().item()
            n += batch[3].shape[0]

            

        if epoch %10 == 0:
            test_acc = evaluate_accuracy(test_iter, net.to(device))
            
            print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
                  f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
    net.eval()
    return net

In [10]:
def preprocess_data(data):
    encoded_input = tokenizer(data.text.tolist(), padding=True, truncation=True, max_length=1000, return_tensors='pt')
    encoded_input = {k: v for k, v in encoded_input.items()}
    encoded_input['labels'] = torch.tensor(data.target.tolist())  
    #model_output = model(**encoded_input)
    return encoded_input


In [24]:
files = ['answers_1.csv','question_zapis.csv']
df = pd.DataFrame()
for file in files:
    df  = pd.concat([df,pd.read_csv(file)])
tar = list(set(df['target']))
chng = {}
for i,j in enumerate(tar):
    chng[j] = i 
df = df.replace({'target':chng})

In [25]:
X_train, X_test = train_test_split(df,shuffle=True,test_size=0.15,random_state=123)

In [26]:
train_data = preprocess_data(X_train)
test_data = preprocess_data(X_test)

In [27]:
train_data['attention_mask'][0].shape

torch.Size([36])

In [31]:
batch_size = 6000
train_dataloader = DataLoader(
    TensorDataset(train_data['input_ids'], train_data['attention_mask'],train_data['token_type_ids'], train_data['labels']),
   
    batch_size = batch_size
)

test_dataloader = DataLoader(
    TensorDataset(test_data['input_ids'], test_data['attention_mask'],test_data['token_type_ids'], test_data['labels']),

    batch_size = batch_size
)


In [ ]:
model = train(model, train_dataloader, test_dataloader, optimizer, 30)

In [62]:
torch.save(model, 'model_all')